<a href="https://colab.research.google.com/github/VanShi87/Spark/blob/main/Exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.3/spark-3.2.3-bin-hadoop2.7.tgz
!tar xf spark-3.2.3-bin-hadoop2.7.tgz

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop2.7"

In [4]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import findspark 
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [8]:
!wget "https://www.gutenberg.org/files/1342/1342-0.txt"

--2022-12-14 10:24:58--  https://www.gutenberg.org/files/1342/1342-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 772145 (754K) [text/plain]
Saving to: ‘1342-0.txt’

1342-0.txt          100%[===================>] 754.05K  --.-KB/s    in 0.1s    

2022-12-14 10:24:59 (5.99 MB/s) - ‘1342-0.txt’ saved [772145/772145]



In [22]:
book = spark.read.text("1342-0.txt")

In [14]:
book.printSchema()
book.dtypes

root
 |-- value: string (nullable = true)



[('value', 'string')]

In [15]:
book.show()

+--------------------+
|               value|
+--------------------+
|The Project Guten...|
|                    |
|This eBook is for...|
|most other parts ...|
|whatsoever. You m...|
|of the Project Gu...|
|www.gutenberg.org...|
|will have to chec...|
|   using this eBook.|
|                    |
|Title: Pride and ...|
|                    |
| Author: Jane Austen|
|                    |
|Release Date: Nov...|
|                    |
|   Language: English|
|                    |
|Produced by: Chuc...|
|             http...|
+--------------------+
only showing top 20 rows



In [29]:
import pyspark.sql.functions as F
lines = book.select(F.split(book.value, " ").alias("line"))
words = lines.select(F.explode(F.col('line')).alias("word"))
words_lower = words.select(F.lower(F.col("word")).alias("word_lower"))
words_clean = words_lower.select(F.regexp_extract(F.col("word_lower"), "[a-z]+", 0).alias("word"))
words_nonull = words_clean.filter(F.col("word") != "")
groups = words_nonull.groupby(F.col("word"))
results = groups.count().orderBy(F.col('count'), ascending=False)


results.coalesce(1).write.csv("./data/simple_count2.csv")

# Новый раздел